# Wrangle and Analyze Data

## Preliminaries

Load packages and libraries required for subsequent blocks.

In [103]:
# import packages
import pandas as pd
import numpy as np
import requests
import tweepy
import json
import os
from timeit import default_timer as timer

## Gather Data

Gather data need for analyzing We Rate Dogs (@dog_rates) twitter feed. There are three files comprising the broader data set.

### File on-hand: Twitter Archive

This file was presented on and downloaded from the Udacity website. So it is simply read into the dataframe using the Pandas 'read_csv' function.

In [104]:
# read dog_rates twitter archive
df = pd.read_csv('twitter-archive-enhanced.csv')

In [37]:
# printing a sample of the dataframe allows for visual inspection.
df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
175,857989990357356544,NaN,NaN,2017-04-28 16:08:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Rosie. She was just informed of the wa...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/857989990...,12,10,Rosie,None,None,None,None
829,768909767477751808,NaN,NaN,2016-08-25 20:35:48 +0000,"<a href=""http://twitter.com/download/iphone"" r...",RT @dog_rates: When it's Janet from accounting...,7.001438e+17,4.196984e+09,2016-02-18 02:24:13 +0000,https://twitter.com/dog_rates/status/700143752...,10,10,None,None,None,pupper,None
992,748692773788876800,NaN,NaN,2016-07-01 01:40:41 +0000,"<a href=""http://twitter.com/download/iphone"" r...",That is Quizno. This is his beach. He does not...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/748692773...,10,10,his,doggo,None,None,None
503,813066809284972545,NaN,NaN,2016-12-25 17:00:08 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tyr. He is disgusted by holiday traffi...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/813066809...,12,10,Tyr,None,None,None,None
2166,669363888236994561,NaN,NaN,2015-11-25 03:56:01 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a Gingivitis Pumpernickel named Z...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/669363888...,10,10,None,None,None,None,None


In [105]:
# using the pandas info() function shows all the variables, their data-types and gives us 
# a count of the entries and missing values.
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

### Download File from Server: Tweet Image Predictions

For the tweet image predictions file, we download the file from a Udacity server using the 'requests'library. This file is then written (saved) to the project directory.

In [19]:
# download tweet image predictions from server - save into workspace
image_prediction = {'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'}
# 'for' loop downloads all files at URL list in ebert_review_urls variable
for url in image_prediction:
    response = requests.get(url)
    with open(os.path.join(url.split('/')[-1]), mode = 'wb') as file:
        file.write(response.content)

We can verify that the download occurred by viewing the project directory to confirm there is an 'image_predictions.csv' file in it

In [106]:
os.listdir()

['stuff.txt',
 'wrangle_act.ipynb',
 'twitter-archive-enhanced.csv',
 '.DS_Store',
 'README.md',
 '.gitignore',
 'tweepy_practice.py',
 '.ipynb_checkpoints',
 'image-predictions.tsv',
 '.git',
 'tweet_json.txt']

We then open the file using the 'read_csv' function with the tab-separator indicated.

In [107]:
df2 = pd.read_csv('image-predictions.tsv',sep='\t')

Next, we perform visual inspection and examine the fields using the info() function, same as with the tweet archive file.

In [108]:
df2.sample(5)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1985,872261713294495745,https://pbs.twimg.com/media/DBrlZk2UQAAfAkd.jpg,2,Labrador_retriever,0.972019,True,flat-coated_retriever,0.008178,True,Chesapeake_Bay_retriever,0.007359,True
340,672248013293752320,https://pbs.twimg.com/media/CVROAIfWsAECA5t.jpg,1,Irish_terrier,0.413173,True,Airedale,0.335616,True,toy_poodle,0.027952,True
1013,709556954897764353,https://pbs.twimg.com/media/CdjaSFCWAAAJZh3.jpg,2,golden_retriever,0.790026,True,kuvasz,0.105031,True,Labrador_retriever,0.087051,True
1486,781955203444699136,https://pbs.twimg.com/media/CtoQGu4XgAQgv5m.jpg,1,pool_table,0.179568,False,dining_table,0.154396,False,microwave,0.033690,False
746,687664829264453632,https://pbs.twimg.com/media/CYsTg1XUsAEPjxE.jpg,1,pug,0.957365,True,French_bulldog,0.038559,True,toy_poodle,0.000667,True


In [109]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


### Create File Dynamically Using Tweepy to Query Twitter API: tweet_json.txt File

The third file needed for this project containing information such as 'retweet' and 'favorite' counts is created dynamically by querying the Twitter api.

First step here took place elsewhere. At apps.twitter.com, I created a (dummy) app which enabled me to create authorization keys as required by Twitter.

Second step is to enter code as indicated in the [Tweepy documentation](http://tweepy.readthedocs.io/en/v3.6.0/getting_started.html#introduction) authenticating access to the API.

In [82]:
# Tweepy auth stuff
# place keys here - delete manually for now

In [110]:
# Tweepy auth stuff continued: note the parameters included in the API call
# setting parser to tweepy.JSONParser ensures that it returns good JSON
# rate_limit parameters enable the function to work through the Twitter rate limit
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth,parser=tweepy.parsers.JSONParser(), wait_on_rate_limit=True,\
                 wait_on_rate_limit_notify=True)

The next step is for Tweepy to access twitter data for the 2356 tweets in the tweet_archive file and to download that data as a json file. First we take the tweet_id variable in the first dataframe (created from the twitter archive csv) and turn it into a list.  The length of that list is returned as a test (2356 is the correct number).

In [87]:
# put all of the tweetids from the twitter_archive csv/dataframe into a list
tweetids = df['tweet_id'].tolist()
# tweetids
len(tweetids)

2356

Then we are going to loop through the list of tweet_ids to access the corresponding records via the Twitter api. We are using a try-except block to save the twitter status for the ids that have records and, separately, to keep a file of the ids which have been deleted. Thanks to the "wait_on_rate_limit" parameter in the api call, the function will not timeout while waiting for new availability from the API.

In [89]:
# List of dictionaries to build and convert to a DataFrame later
tweet_list = []
tweet_errors = []
start = timer()
for tweet in tweetids:
    try:
        full_status = api.get_status(tweet, tweet_mode='extended')
        tweet_list.append(full_status)
    except Exception as e:
        end = timer()
        print('No status found for tweetid ' + str(tweet))
        print('Time elapsed to error ' + str(end - start))
        tweet_errors.append(tweet)
print('Total runtime for function ' + str((end - start)/60) + ' minutes.')
print(len(tweet_list))
print (len(tweet_errors))

No status found for tweetid 888202515573088257
Time elapsed to error 4.967670181998983
No status found for tweetid 873697596434513921
Time elapsed to error 21.60596933600027
No status found for tweetid 869988702071779329
Time elapsed to error 26.490966256998945
No status found for tweetid 866816280283807744
Time elapsed to error 29.46525864698924
No status found for tweetid 861769973181624320
Time elapsed to error 34.22458414500579
No status found for tweetid 845459076796616705
Time elapsed to error 53.73645277399919
No status found for tweetid 842892208864923648
Time elapsed to error 56.511891781992745
No status found for tweetid 837012587749474308
Time elapsed to error 65.38332627300406
No status found for tweetid 827228250799742977
Time elapsed to error 83.12681407199125
No status found for tweetid 802247111496568832
Time elapsed to error 122.45716445101425
No status found for tweetid 775096608509886464
Time elapsed to error 169.60564937099116


Rate limit reached. Sleeping for: 468
Rate limit reached. Sleeping for: 708


Time total runtime for function 2.826760822849853 minutes.
2345
11


There were 2345 tweets that had meta-data returned and 11 which had been deleted subsequent to the creation of the twitter_archive csv.  Next let's take a look at one line from the tweet_list to see what the data looks like.

In [90]:
tweet_list[:1]

[{'contributors': None,
  'coordinates': None,
  'created_at': 'Tue Aug 01 16:23:56 +0000 2017',
  'display_text_range': [0, 85],
  'entities': {'hashtags': [],
   'media': [{'display_url': 'pic.twitter.com/MgUWQ76dJU',
     'expanded_url': 'https://twitter.com/dog_rates/status/892420643555336193/photo/1',
     'id': 892420639486877696,
     'id_str': '892420639486877696',
     'indices': [86, 109],
     'media_url': 'http://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
     'media_url_https': 'https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg',
     'sizes': {'large': {'h': 528, 'resize': 'fit', 'w': 540},
      'medium': {'h': 528, 'resize': 'fit', 'w': 540},
      'small': {'h': 528, 'resize': 'fit', 'w': 540},
      'thumb': {'h': 150, 'resize': 'crop', 'w': 150}},
     'type': 'photo',
     'url': 'https://t.co/MgUWQ76dJU'}],
   'symbols': [],
   'urls': [],
   'user_mentions': []},
  'extended_entities': {'media': [{'display_url': 'pic.twitter.com/MgUWQ76dJU',
     'expanded_url': 'htt

This appears to be well-formed JSON:we'll know for sure when we save it and try to reopen that file.  Next step is to store the output as a text file 'tweet_json.txt' using the json.dump function.

In [91]:
# Store tweet info in JSON format

with open('tweet_json.txt', 'w') as outfile:  
    json.dump(tweet_list, outfile)

Our test to make sure it worked is to see if there is a 'tweet_json.txt' file in our working directory.

In [111]:
os.listdir()

['stuff.txt',
 'wrangle_act.ipynb',
 'twitter-archive-enhanced.csv',
 '.DS_Store',
 'README.md',
 '.gitignore',
 'tweepy_practice.py',
 '.ipynb_checkpoints',
 'image-predictions.tsv',
 '.git',
 'tweet_json.txt']

And it is there! Our next step is to read this text file into a dataframe using Pandas read_json function

In [112]:
df_json = pd.read_json(open('tweet_json.txt'))

In [113]:
df_json.head(1)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,geo,...,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,retweet_count,retweeted,retweeted_status,source,truncated,user
0,NaN,NaN,2017-08-01 16:23:56,"[0, 85]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'media': [{'id': 892420639486877696, 'id_str'...",38693,False,This is Phineas. He's a mystical boy. Only eve...,NaN,...,NaN,NaN,NaN,NaN,8559,False,NaN,"<a href=""http://twitter.com/download/iphone"" r...",False,"{'id': 4196983835, 'id_str': '4196983835', 'na..."


This is a massive file with a ton of data, so I only printed out the first item to make sure that the dataframe is properly formed. The next (and *final*) step in gathering our data is to select only the variables needed for this project ('id','retweet_count','favorite_count') and put them into a new dataframe.

In [114]:
df3 = df_json[['id','retweet_count','favorite_count']]

In [115]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2345 entries, 0 to 2344
Data columns (total 3 columns):
id                2345 non-null int64
retweet_count     2345 non-null int64
favorite_count    2345 non-null int64
dtypes: int64(3)
memory usage: 73.3 KB


In [116]:
df3.head(3)

,id,retweet_count,favorite_count
0,892420643555336193,8559,38693
1,892177421306343426,6294,33167
2,891815181378084864,4176,24967


We can see that the dataframe for these items is complete and tidy.  Now on to further assessment of the data we have gathered in these three files.

## Assess Data

Let's start by visually inspecting each of the three dataframes created from the three files.

In [118]:
df.sample(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1791,677335745548390400,NaN,NaN,2015-12-17 03:53:20 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",Downright inspiring 12/10 https://t.co/vSLtYBWHcQ,NaN,NaN,NaN,https://vine.co/v/hbLbH77Ar67,12,10,None,None,None,None,None
1603,685943807276412928,NaN,NaN,2016-01-09 21:58:42 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is the newly formed pupper a capella grou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/685943807...,8,10,the,None,None,pupper,None
64,879674319642796034,8.795538e+17,3.105441e+09,2017-06-27 12:14:36 +0000,"<a href=""http://twitter.com/download/iphone"" r...",@RealKentMurphy 14/10 confirmed,NaN,NaN,NaN,NaN,14,10,None,None,None,None,None
1813,676776431406465024,NaN,NaN,2015-12-15 14:50:49 +0000,"<a href=""http://twitter.com/download/iphone"" r...","When someone yells ""cops!"" at a party and you ...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/676776431...,10,10,None,None,None,None,None
1716,680206703334408192,NaN,NaN,2015-12-25 02:01:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I hope everyone enjoys this picture as much as...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680206703...,12,10,Toby,None,None,None,None
